In [ ]:
import torch
import matplotlib.pyplot as plt
import numpy as np

## Create data

In [ ]:
np.random.seed(0)
xs1= np.random.randn(500)*100+350
ys1 = 1.5*xs1+0.5+np.random.randn(500)*50
xs2 = np.random.randn(200)*150 + 700
ys2 = 0.5*xs2+500+np.random.randn(200)*50
xs3 = np.random.randn(200)*200 + 1250
ys3 = 0.2*xs3+800+np.random.randn(200)*50
xs = np.concatenate((xs1,xs2,xs3))
ys = np.concatenate((ys1,ys2,ys3))

In [ ]:
plt.xlabel('area')
plt.ylabel('price')
plt.scatter(xs,ys)
plt.show()